<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# PySpark LLM Inference: Gemma-7b

In this notebook, we demonstrate distributed inference with the Google [Gemma-7b-instruct](https://huggingface.co/google/gemma-7b-it) LLM, using open-weights on Huggingface.

The Gemma-7b-instruct is an instruction-fine-tuned version of the Gemma-7b base model.

**Note:** Running this model on GPU with 16-bit precision requires **~18 GB** of GPU RAM. Make sure your instances have sufficient GPU capacity.

In [1]:
import os

# Check the cluster environment to handle any platform-specific configurations.
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)
on_dataproc = os.environ.get("DATAPROC_IMAGE_VERSION", False)
on_standalone = not (on_databricks or on_dataproc)

In [2]:
# Set the huggingface cache dir to dbfs/gcs so that executors can load the model from cache.
if on_databricks:
    hf_home = "/dbfs/FileStore/hf-home"
    dbutils.fs.mkdirs(hf_home)
    os.environ["HF_HOME"] = hf_home
elif on_dataproc:
    hf_home = "/mnt/gcs/hf-home"
    os.mkdir(hf_home) if not os.path.exists(hf_home) else None
    os.environ["HF_HOME"] = hf_home

### Warmup: Running locally

**Note**: If the driver node does not have sufficient GPU capacity, proceed to the PySpark section.

First visit the [Gemma Huggingface repository](https://huggingface.co/google/gemma-7b-it) to accept the terms to access the model, then login via huggingface_hub.

In [ ]:
from huggingface_hub import login

login()

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it",
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
input_text = "Write me a poem about Apache Spark."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=100, temperature=0.1, do_sample=True)
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Apache Spark.

In the realm of big data, a spark ignites,
A framework born to conquer the night.
Apache Spark, a lightning-fast tool,
For processing data, swift and cool.

With its resilient distributed architecture,
It slices through terabytes with grace.
No longer bound by memory's plight,
Spark empowers us to analyze with might.

From Python to Scala, it's a versatile spark,
Unveiling insights hidden in the dark.



In [4]:
import torch

del model
torch.cuda.empty_cache()

## PySpark

In [28]:
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.functions import predict_batch_udf

In [6]:
import os
import pandas as pd
import datasets
from datasets import load_dataset
datasets.disable_progress_bars()

#### Create Spark Session

For local standalone clusters, we'll connect to the cluster and create the Spark Session.  
For CSP environments, Spark will either be preconfigured (Databricks) or we'll need to create the Spark Session (Dataproc).

In [ ]:
conf = SparkConf()

if 'spark' not in globals():
    if on_standalone:
        import socket
        conda_env = os.environ.get("CONDA_PREFIX")
        hostname = socket.gethostname()
        conf.setMaster(f"spark://{hostname}:7077")
        conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
        conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
        # Point PyTriton to correct libpython3.11.so:
        conf.set("spark.executorEnv.LD_LIBRARY_PATH", f"{conda_env}/lib:{conda_env}/lib/python3.11/site-packages/nvidia_pytriton.libs:$LD_LIBRARY_PATH")
    elif on_dataproc:
        # Point PyTriton to correct libpython3.11.so:
        conda_lib_path="/opt/conda/miniconda3/lib"
        conf.set("spark.executorEnv.LD_LIBRARY_PATH", f"{conda_lib_path}:$LD_LIBRARY_PATH")
        conf.set("spark.executor.instances", "4") # dataproc defaults to 2

    conf.set("spark.executor.cores", "8")
    conf.set("spark.task.maxFailures", "1")
    conf.set("spark.task.resource.gpu.amount", "0.125")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
    conf.set("spark.python.worker.reuse", "true")

conf.set("spark.executorEnv.HF_HOME", hf_home)
conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

25/02/09 21:12:14 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
25/02/09 21:12:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/09 21:12:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Load and preprocess DataFrame

Load the code exercises dataset from Huggingface and add a prompt prefix. 

In [37]:
# dataset = load_dataset("imbue/code-comprehension", split="train[:1%]")
dataset = load_dataset("jinaai/code_exercises", split="train[:1%]")
dataset = dataset.to_pandas()["problem"]

In [38]:
df = spark.createDataFrame(dataset, schema=StringType()).withColumnRenamed("value", "prompt").limit(100)

In [39]:
prefix = "Implement this function:\n"
df = df.withColumn('prompt', concat(lit(prefix), col('prompt'))).select("prompt")

In [40]:
df.show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                              prompt|
+----------------------------------------------------------------------------------------------------+
|Implement this function:\ndef calculate_average_price(prices):\n    """\n    Calculate the averag...|
|Implement this function:\ndef find_gene(sequence, target_gene):\n\n"""\n\nGiven a DNA sequence an...|
|Implement this function:\ndef get_common_words(string1, string2):\n    """\n    This function tak...|
|Implement this function:\ndef advantages_equal(musician_1, musician_2):\n\n    """This exercise w...|
|Implement this function:\ndef create_tuple(n):\n    """\n    Given an integer n, create a nested ...|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [41]:
print(df.take(1)[0].prompt)

Implement this function:
def calculate_average_price(prices):
    """
    Calculate the average price of a list of fashion items.

    Args:
    prices (list): A list of prices of fashion items.

    Returns:
    float: The average price of the fashion items.
    """


In [42]:
df.write.mode("overwrite").json("spark-dl-datasets/code_exercises")

## Using Triton Inference Server
In this section, we demonstrate integration with the [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), an open-source, GPU-accelerated serving solution for DL.  
We use [PyTriton](https://github.com/triton-inference-server/pytriton), a Flask-like framework that handles client/server communication with the Triton server.  

The process looks like this:
- Distribute a PyTriton task across the Spark cluster, instructing each node to launch a Triton server process.
- Define a Triton inference function, which contains a client that binds to the local server on a given node and sends inference requests.
- Wrap the Triton inference function in a predict_batch_udf to launch parallel inference requests using Spark.
- Finally, distribute a shutdown signal to terminate the Triton server processes on each node.

<img src="../images/spark-pytriton.png" alt="drawing" width="700"/>

In [43]:
from functools import partial

Import the helper class from pytriton_utils.py:

In [44]:
sc.addPyFile("pytriton_utils.py")

from pytriton_utils import TritonServerManager

Define the Triton Server function:

In [70]:
def triton_server(ports):
    import time
    import signal
    import numpy as np
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM
    from pytriton.decorators import batch
    from pytriton.model_config import DynamicBatcher, ModelConfig, Tensor
    from pytriton.triton import Triton, TritonConfig
    from pyspark import TaskContext

    print(f"SERVER: Initializing model on worker {TaskContext.get().partitionId()}.")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
    model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", device_map="auto", torch_dtype=torch.bfloat16)
    print(f"SERVER: Using {device} device.")

    @batch
    def _infer_fn(**inputs):
        prompts = np.squeeze(inputs["prompts"]).tolist()
        print(f"SERVER: Received batch of size {len(prompts)}")
        decoded_prompts = [p.decode("utf-8") for p in prompts]
        input_ids = tokenizer(decoded_prompts, padding=True, return_tensors="pt").to(device)
        output_ids = model.generate(**input_ids, max_new_tokens=256, temperature=0.1, do_sample=True)
        responses = np.array([[tokenizer.decode(o)] for o in output_ids])
        return {
            "responses": responses,
        }

    workspace_path = f"/tmp/triton_{time.strftime('%m_%d_%M_%S')}"
    triton_conf = TritonConfig(http_port=ports[0], grpc_port=ports[1], metrics_port=ports[2])
    with Triton(config=triton_conf, workspace=workspace_path) as triton:
        triton.bind(
            model_name="gemma-7b",
            infer_func=_infer_fn,
            inputs=[
                Tensor(name="prompts", dtype=object, shape=(-1,)),
            ],
            outputs=[
                Tensor(name="responses", dtype=object, shape=(-1,)),
            ],
            config=ModelConfig(
                max_batch_size=16,
                batcher=DynamicBatcher(max_queue_delay_microseconds=5000),  # 5ms
            ),
            strict=True,
        )

        def _stop_triton(signum, frame):
            print("SERVER: Received SIGTERM. Stopping Triton server.")
            triton.stop()

        signal.signal(signal.SIGTERM, _stop_triton)

        print("SERVER: Serving inference")
        triton.serve()

#### Start Triton servers

**Specify the number of nodes in the cluster.**  
Following the README, the example standalone cluster uses 1 node. The example Databricks/Dataproc cluster scripts use 4 nodes by default. 

In [71]:
# Change based on cluster setup
num_nodes = 1 if on_standalone else 4

The `TritonClusterManager` will handle the lifecycle of Triton server instances across the Spark cluster:
- Find available ports for HTTP/gRPC/metrics
- Deploy a server on each node via stage-level scheduling
- Gracefully shutdown servers across nodes

In [72]:
model_name = "gemma-7b"
server_manager = TritonServerManager(num_nodes=num_nodes, model_name=model_name)

In [73]:
# Returns {'hostname', (server_pid, [http_port, grpc_port, metrics_port])}
server_manager.start_servers(triton_server)

2025-02-09 21:44:40,454 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-09 21:44:40,456 - INFO - Starting 1 servers.


{'cb4ae00-lcedt': (4050539, [7000, 7001, 7002])}

#### Define client function

Get the hostname -> url mapping from the server manager:

In [74]:
host_to_grpc_url = server_manager.host_to_grpc_url  # or server_manager.host_to_http_url

In [75]:
def triton_fn(model_name, host_to_url):
    import socket
    import numpy as np
    from pytriton.client import ModelClient

    url = host_to_url[socket.gethostname()]
    print(f"Connecting to Triton model {model_name} at {url}.")

    def infer_batch(inputs):
        with ModelClient(url, model_name, inference_timeout_s=500) as client:
            flattened = np.squeeze(inputs).tolist()
            # Encode batch
            encoded_batch = [[text.encode("utf-8")] for text in flattened]
            encoded_batch_np = np.array(encoded_batch, dtype=np.bytes_)
            # Run inference
            result_data = client.infer_batch(encoded_batch_np)
            result_data = np.squeeze(result_data["responses"], -1)
            return result_data
        
    return infer_batch

In [76]:
generate = predict_batch_udf(partial(triton_fn, model_name=model_name, host_to_url=host_to_grpc_url),
                             return_type=StringType(),
                             input_tensor_shapes=[[1]],
                             batch_size=4)

#### Load and preprocess DataFrame

We'll parallelize over a small set of questions for demonstration.

In [77]:
df = spark.read.json("spark-dl-datasets/code_exercises").limit(32).repartition(8)

#### Run Inference

In [78]:
%%time
# first pass caches model/fn
preds = df.withColumn("response", generate(col("prompt")))
results = preds.collect()

CPU times: user 14.4 ms, sys: 9.83 ms, total: 24.3 ms
Wall time: 23.9 s


In [79]:
%%time
preds = df.withColumn("response", generate("prompt"))
results = preds.collect()

CPU times: user 6.76 ms, sys: 9.73 ms, total: 16.5 ms
Wall time: 23.2 s


In [83]:
print(f"Q: {results[2].prompt} \n")
print(f"A: {results[2].response} \n")

Q: Implement this function:
def update_actor_details(actor_data):
    """
    This function takes a list of dictionaries containing actor/actress details. It updates the salary of each actor
    based on their experience and returns the updated list.

    Args:
    - actor_data (list of dictionaries): A list containing actor/actress details. Each dictionary contains 'name',
    'experience', and 'salary' keys.

    Returns:
    - updated_actor_data (list of dictionaries): The updated list of actor/actress details with their updated salary.
    """ 

A: <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

#### Shut down server on each executor

In [84]:
server_manager.stop_servers()

2025-02-09 21:47:54,974 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)


2025-02-09 21:48:00,214 - INFO - Sucessfully stopped 1 servers.                 


[True]

In [85]:
if not on_databricks: # on databricks, spark.stop() puts the cluster in a bad state
    spark.stop()